# Exercise 1 : Association Rule Mining

Extract frequently occuring words, phrases and association among them for the given sentences:<br>
'For playing football you need sports shoes',<br>
'Football should not be played without shoes',<br>
'Football is my favourite sport',<br>
"Don't play football wearing sneakers",<br>
'Sneakers are fashionable shoes'<br>

In [111]:
#!pip install mlxtend

In [6]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')

#adding individual printable characters to list of wtop words so that they get renoved along with the stopwords
stop_words = stop_words + list(string.printable)
lemmatizer = WordNetLemmatizer()

In [7]:
text_data_df = pd.DataFrame({'text' : ['For playing football you need sports shoes',\
'Football should not be played without shoes',\
'Football is my favourite sport',\
"Don't play football wearing sneakers",\
'Sneakers are fashionable shoes']})

In [8]:
text_data_df['cleaned_text'] = text_data_df['text'].apply(\
lambda x : [lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x))) if word.lower() not in stop_words])
text_data_df

,text,cleaned_text
0,For playing football you need sports shoes,"[playing, football, need, sport, shoe]"
1,Football should not be played without shoes,"[football, played, without, shoe]"
2,Football is my favourite sport,"[football, favourite, sport]"
3,Don't play football wearing sneakers,"[play, football, wearing, sneaker]"
4,Sneakers are fashionable shoes,"[sneaker, fashionable, shoe]"


In [10]:
te = TransactionEncoder()
te_ary = te.fit(text_data_df['cleaned_text']).transform(text_data_df['cleaned_text'])
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.8,(football)
1,0.6,(shoe)
2,0.4,(sneaker)
3,0.4,(sport)
4,0.4,"(football, shoe)"
5,0.4,"(sport, football)"


In [11]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(football),(shoe),0.8,0.6,0.4,0.500000,0.833333,-0.08,0.800000
1,(shoe),(football),0.6,0.8,0.4,0.666667,0.833333,-0.08,0.600000
2,(sport),(football),0.4,0.8,0.4,1.000000,1.250000,0.08,inf
3,(football),(sport),0.8,0.4,0.4,0.500000,1.250000,0.08,1.200000


In [12]:
#For more on association rule mining visit: 
#http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/